In [5]:
import os
import h5py
import sys


import pandas as pd
import geopandas as gpd
import math
import pdal
from osgeo import ogr
from shapely.geometry import MultiLineString, LineString, Point, Polygon
from shapely import wkt

# for plotting
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import fiona

# for ALS
import subprocess
import json
from shapely.ops import transform

## 1. Importing ATL08 data from local computer and reading it into geopandas dataframe

Steps taken:
1. Loop over files in directory that start with 'ATL08_' and end with '.h5'. Here only few files are actually used 
2. Loop over the data points inside the file
3. If the data points have coordinates that fall inside the bounding box of Estonia, save them into a list

This is still quite unfinished. At the moment I save the solar elevation and the night flag together with the coordinates. However, there are several other steps I need to add here.
For example, from my P2
* reject canopy heights smaller than 0 m and greater than 50 m
* the segments with radiometric parameter values exceeding 16 photons per shot will be eliminated as the ATLAS detector can only detect 16 photons per outgoing shot.
* using the height difference between the estimated ground surface by ICESat-2 and the MERIT DEM used by the ICESat-2 systems, segments where the absolute height difference is greater than 30 meters will be considered as noise and filtered out.
* To detect scattering, msq flag from ATL09 product can be used where value larger than 0 indicates scattering which may affect the results. (I might leave this to later stage)
* the signal return strength can vary due to snow (Neuenschwander et al., 2022), therefore the transects acquired in the presence of snow are filtered out by using ATL08 snow flag.
* the date - I need to know the time difference between the icesat and als data

Also, the id-s of the transect lines are not yet saved.

In [6]:
data_loc = 'Z:\\Thesis\\Data\\ATL08_data\\'
iterations = 0
# list where to add data - later used to create a dataframe
dataframe = [] 
tracks = ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']

for file in os.listdir(data_loc):
    # this is to limit the data while still developing
    if iterations < 5:
        if file.startswith('ATL08_') and file.endswith('.h5'):
            #with h5py.File(FILE_NAME, mode='r') as f:
            FILE_NAME = data_loc+file
            print("\n", file)
        
            with h5py.File(FILE_NAME, mode='r') as f:
                
                # skip the file in case the quality assessment is fail - UNFINISHED
                #if f['/quality_assessment']
                
                # Here add a method to iterate over all of the 6 tracks
                
                # Get the data on solar elevation and night flag ready
                solar_all = f['/gt1l/land_segments/solar_elevation']
                solar = solar_all[:]
                night_all = f['/gt1l/land_segments/night_flag']
                night = night_all[:]
                
                # Get the coordinates
                latvar = f['/gt1l/land_segments/latitude']
                lat = latvar[:]
                lonvar = f['/gt1l/land_segments/longitude']
                lon = lonvar[:]
                dset_name = '/gt1l/land_segments/dem_h'
                datavar = f[dset_name]
                data = datavar[:]
                units = datavar.attrs['units']
                long_name = datavar.attrs['long_name']
                _FillValue = datavar.attrs['_FillValue']
                
                inner_iter = 0
                
                # loop over the data points and if inside bbox of Estonia, add them to the dataframe
                while inner_iter < lat.size:
                    # Estonia's bbox is 'EE': ('Estonia', (23.3397953631, 57.4745283067, 28.1316992531, 59.6110903998))
                    # Use if statement to filter out data outside the bbox
                    if lat[inner_iter]<59.6110903998 and lat[inner_iter] > 57.4745283067 and lon[inner_iter] <28.1316992531 and lon[inner_iter]>23.3397953631:
                        point = Point(lon[inner_iter], lat[inner_iter])
                        dataframe.append({'lat': lat[inner_iter], 'lon': lon[inner_iter], 'geometry': point, 'data': data[inner_iter], 'night': night[inner_iter], 'solar_el': solar[inner_iter]})
                    inner_iter += 1

                iterations += 1
    


 ATL08_20181014020411_02360105_005_01.h5

 ATL08_20181014020936_02360106_005_01.h5

 ATL08_20181016132807_02740102_005_01.h5

 ATL08_20181016133637_02740103_005_01.h5

 ATL08_20181018015550_02970105_005_01.h5


In [8]:
df = pd.DataFrame(dataframe)
df.head()

,lat,lon,geometry,data,night,solar_el
0,59.609509,24.907209,POINT (24.907209396362305 59.6095085144043),17.497229,1,-21.437059
1,59.608612,24.907028,POINT (24.907028198242188 59.608612060546875),17.498577,1,-21.437450
2,59.607719,24.906845,POINT (24.906845092773438 59.60771942138672),17.499891,1,-21.437843
3,59.606823,24.906666,POINT (24.906665802001953 59.6068229675293),17.501179,1,-21.438231
4,59.605930,24.906487,POINT (24.90648651123047 59.60593032836914),17.502453,1,-21.438620


In [9]:
# put this into geodataframe
gdf = gpd.GeoDataFrame(data = dataframe, geometry =gpd.points_from_xy(df.lon, df.lat), crs= 'EPSG:4326')
gdf

,lat,lon,geometry,data,night,solar_el
0,59.609509,24.907209,POINT (24.90721 59.60951),17.497229,1,-21.437059
1,59.608612,24.907028,POINT (24.90703 59.60861),17.498577,1,-21.437450
2,59.607719,24.906845,POINT (24.90685 59.60772),17.499891,1,-21.437843
3,59.606823,24.906666,POINT (24.90667 59.60682),17.501179,1,-21.438231
4,59.605930,24.906487,POINT (24.90649 59.60593),17.502453,1,-21.438620
...,...,...,...,...,...,...
1381,59.547535,24.115896,POINT (24.11590 59.54753),18.762743,1,-23.999195
1382,59.542168,24.114811,POINT (24.11481 59.54217),18.767702,1,-24.001675
1383,59.537701,24.113901,POINT (24.11390 59.53770),18.770588,1,-24.003754
1384,59.529652,24.112267,POINT (24.11227 59.52965),18.773706,1,-24.007467


Reproject the geodataframe with ATL-08 data to Estonian crs.

In [10]:
gdf_est = gdf.to_crs('epsg:3301')
# also reassign the lat and long columns
gdf_est['lon'] = gdf_est['geometry'].x
gdf_est['lat'] = gdf_est['geometry'].y
gdf_est

,lat,lon,geometry,data,night,solar_el
0,6.608361e+06,551219.590202,POINT (551219.590 6608361.451),17.497229,1,-21.437059
1,6.608261e+06,551210.711176,POINT (551210.711 6608261.445),17.498577,1,-21.437450
2,6.608162e+06,551201.718180,POINT (551201.718 6608161.863),17.499891,1,-21.437843
3,6.608062e+06,551192.945760,POINT (551192.946 6608061.858),17.501179,1,-21.438231
4,6.607962e+06,551184.167062,POINT (551184.167 6607962.279),17.502453,1,-21.438620
...,...,...,...,...,...,...
1381,6.601116e+06,506555.441306,POINT (506555.441 6601116.159),18.762743,1,-23.999195
1382,6.600518e+06,506495.078063,POINT (506495.078 6600518.085),18.767702,1,-24.001675
1383,6.600020e+06,506444.453731,POINT (506444.454 6600020.329),18.770588,1,-24.003754
1384,6.599123e+06,506353.470178,POINT (506353.470 6599123.437),18.773706,1,-24.007467


In [11]:
# save the geodataframe in geopackage to open in QGIS
gdf_est.to_file('dataframe_est.gpkg', driver='GPKG', layer='ALS_08_est')

C:\Users\Kati\anaconda3\envs\thesis\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


I don't think this put the elevation data to the right crs - still need to fix this.

## 2. Get bounding boxes for each ATL08 point
For each ATL08 data point, a bounding box is needed for clipping ALS point cloud transect. ATL08 points are about 100 meters apart from each other, which gives 50 meters both ways from a data point along the transect. The size of each ICESat-2 footprint on the ground has been determined to range between 10–12 m in diameter, therefore the bounding box is 12 meters wide.

What I am doing is finding points 6 meters from the ATL points at the right angle with the transect line.
THe implementation came largely from here: https://glenbambrick.com/tag/shapely/

In a function, for each ICESat-2 point:

* Find the point 50 m towards the next point
* Find two points that are at 90 degrees from the line connecting the IceSAT point and the point 50m away
* Find the previous icesat point and a point halfway towards there
* Find the two points at 90 degrees from that point
* Save the four points as Polygon to the dataframe

There is a mistake here - it does not regard the scan lines, so it just takes the previous and next from the dataframe, which is not correct.
First and last data point from the scan line should be disregaded - no bounding box.

There is some repetition of data here - if two IceSAT points are 100 m from each other then they share two of their bbox corners. However, the points are often missing or filtered out, therefore can't rely on saving the points only once.

In [13]:
# first helping funcitions
def getAngle(pt1, pt2):
    x_diff = pt2.x - pt1.x
    y_diff = pt2.y - pt1.y
    return math.degrees(math.atan2(y_diff, x_diff))


## get the first point at the right angle from the icesat transect
def getPoint1(pt, bearing, dist):
    angle = bearing + 90
    bearing = math.radians(angle)
    x = pt.x + dist * math.cos(bearing)
    y = pt.y + dist * math.sin(bearing)
    return Point(x, y)

## get the second  point at the right angle from the icesat transect - to the other side of the transect
def getPoint2(pt, bearing, dist):
    bearing = math.radians(bearing)
    x = pt.x + dist * math.cos(bearing)
    y = pt.y + dist * math.sin(bearing)
    return Point(x, y)


# assign dataframe here - just to leave me an option to use another dataframe. Probably should write this into a function.
dataframe = gdf_est

# iterate over icesat points
for index, row in dataframe.iterrows():
    # for now - ignore the first and the last point
    if index == 0 or index == len(dataframe)-1:
        continue
    else:  
        # create a line to the previous point and to the next icesat point
        line_before = LineString([dataframe['geometry'][index-1], dataframe['geometry'][index]])
        line_next = LineString([dataframe['geometry'][index], dataframe['geometry'][index+1]])
        
        # find a point to mid-way (50 m) to the previous point and to the next point
        interpolation_bef = line_before.interpolate(50)
        interpolation_next = line_next.interpolate(50)
        
        # first find the two points 90 degrees from the line between the point  and the one half way to the previous point
        angle = getAngle(interpolation_bef, dataframe['geometry'][index-1])
        # create point at 90 degrees angle 6 meters from the transect line
        end_1 = getPoint1(interpolation_bef, angle, 6)
        # and to the other side
        angle = getAngle(end_1, interpolation_bef)
        end_2 = getPoint2(interpolation_bef, angle, 6)
        
        # now two points towards the next point
        angle = getAngle(interpolation_next, dataframe['geometry'][index+1])
        end_3 = getPoint1(interpolation_next, angle, 6)
        angle = getAngle(end_3, interpolation_next)
        end_4 = getPoint2(interpolation_next, angle, 6)
        
        # now store this as Polygon  -this doesnt open in qgis. Try to print out the points to see if they are in the right order
        # make sure you dont have crossing edges
        poly = Polygon([(end_1.x, end_1.y), (end_2.x, end_2.y), (end_4.x, end_4.y), (end_3.x, end_3.y), (end_1.x, end_1.y)])
        dataframe.loc[index, 'polygon']=poly

        

In [14]:
gdf_est

,lat,lon,geometry,data,night,solar_el,polygon
0,6.608361e+06,551219.590202,POINT (551219.590 6608361.451),17.497229,1,-21.437059,NaN
1,6.608261e+06,551210.711176,POINT (551210.711 6608261.445),17.498577,1,-21.437450,"POLYGON ((551209.1918591695 6608312.177747834,..."
2,6.608162e+06,551201.718180,POINT (551201.718 6608161.863),17.499891,1,-21.437843,"POLYGON ((551200.2384415013 6608212.187494098,..."
3,6.608062e+06,551192.945760,POINT (551192.946 6608061.858),17.501179,1,-21.438231,"POLYGON ((551191.3718946896 6608112.578340041,..."
4,6.607962e+06,551184.167062,POINT (551184.167 6607962.279),17.502453,1,-21.438620,"POLYGON ((551182.5780811904 6608012.578400162,..."
...,...,...,...,...,...,...,...
1381,6.601116e+06,506555.441306,POINT (506555.441 6601116.159),18.762743,1,-23.999195,POLYGON ((506685.77842671686 6602461.259593124...
1382,6.600518e+06,506495.078063,POINT (506495.078 6600518.085),18.767702,1,-24.001675,POLYGON ((506544.45067242533 6601067.014104795...
1383,6.600020e+06,506444.453731,POINT (506444.454 6600020.329),18.770588,1,-24.003754,POLYGON ((506484.04969997925 6600468.948769309...
1384,6.599123e+06,506353.470178,POINT (506353.470 6599123.437),18.773706,1,-24.007467,"POLYGON ((506433.4381055149 6599971.189830648,..."


## Clip ALS using the polygons created around each ATL08 point

This may not be the most efficient solution.
I am first creating another dataframe for the ALS files I have and save their bounding boxes.
Later I will use this to find an intersection with the bounding boxes of the ICESat bbox-s.

In [15]:
# iterate over files in that directory to get the ALS files - right now I only had 2 which I knew had an intersection with 
# the small ICESat-2 data I am using here
ALS_data = '..\\Data\\ALS\\'
file_names = []
# list to save the ALS bbox
bboxes = []


# this is to return 2d element (als bbox) from 3d
def _to_2d(x, y, z):
    return tuple(filter(None, [x, y]))

# I need to double check where I adapted this solution from 
for filename in os.listdir(ALS_data):
    if filename.endswith('.las'):
        f = os.path.join(ALS_data, filename)
        # checking if it is a file
        if os.path.isfile(f):
            
            result = subprocess.run(['pdal', 'info', f],
                        stderr = subprocess.PIPE,  # stderr and stdout get
                        stdout = subprocess.PIPE)  # captured as bytestrings
            json_result = json.loads(result.stdout.decode())

            coords = json_result['stats']['bbox']['native']['boundary']['coordinates']
            bbox = transform(_to_2d, Polygon(*coords))
            # use transform to drop the Z coordinate
            file_names.append(filename)
            bboxes.append(bbox)
        


In [16]:
# create dataframe for ALS files
als_df = pd.DataFrame(
    {'File': file_names,
     'BBOX': bboxes})
als_gdf = gpd.GeoDataFrame(als_df, geometry=als_df['BBOX'])
als_gdf

,File,BBOX,geometry
0,joesuu.las,"POLYGON ((519000.01 6590000, 519000.01 6590999...","POLYGON ((519000.010 6590000.000, 519000.010 6..."
1,naage_tee.las,"POLYGON ((519000 6586000, 519000 6586999.99, 5...","POLYGON ((519000.000 6586000.000, 519000.000 6..."
2,sambliku_1.las,"POLYGON ((529000 6589000, 529000 6589846.24, 5...","POLYGON ((529000.000 6589000.000, 529000.000 6..."
3,sambliku_2.las,"POLYGON ((528000 6589000, 528000 6589999.99, 5...","POLYGON ((528000.000 6589000.000, 528000.000 6..."


Now iterate over the dataframe holding icesat data and check if either of the bbox has intersection with the ALS. If it does, save the index of the ALS to a list.

This for each icesat point iterates over all of the ALS files, so the solution is not scalable.

In [17]:
# add new column and first add a no data as value to it - it made it easier later to find the icesat points that did have intersect
gdf_est['ALS'] = 'nan'
gdf_est.head()

,lat,lon,geometry,data,night,solar_el,polygon,ALS
0,6.608361e+06,551219.590202,POINT (551219.590 6608361.451),17.497229,1,-21.437059,NaN,nan
1,6.608261e+06,551210.711176,POINT (551210.711 6608261.445),17.498577,1,-21.437450,"POLYGON ((551209.1918591695 6608312.177747834,...",nan
2,6.608162e+06,551201.718180,POINT (551201.718 6608161.863),17.499891,1,-21.437843,"POLYGON ((551200.2384415013 6608212.187494098,...",nan
3,6.608062e+06,551192.945760,POINT (551192.946 6608061.858),17.501179,1,-21.438231,"POLYGON ((551191.3718946896 6608112.578340041,...",nan
4,6.607962e+06,551184.167062,POINT (551184.167 6607962.279),17.502453,1,-21.438620,"POLYGON ((551182.5780811904 6608012.578400162,...",nan


In [18]:
# iterate over icesat data
for iindex, row in gdf_est.iterrows():
    # if there is no polygon saved (eg for first and last points) then ignore
    if pd.isna(row['polygon']):
        gdf_est.at[iindex, 'ALS'] = None
        continue
    # save the polygon to variable
    icesat_polygon = wkt.loads(str(row['polygon']))
    
    # this is a list holding the indices of the ALS df rows with which there is an intersection
    # one icesat bbox can intersect with more than one als file
    als_files = []

    # iterate over all the als dataframes
    for index, row in als_gdf.iterrows():
        # save als bbox to polygon variable
        als_polygon = wkt.loads(str(row['BBOX'])) 

        # if the icesat and als polygons intersect, then save als index
        if icesat_polygon.intersects(als_polygon):
            als_files.append(index)
    
    # once all als has been iterated for this icesat
    # if there were intersections found, add the list to df, if not add none value
    if len(als_files) > 0:
        gdf_est.at[iindex, 'ALS'] = als_files
    else:
        gdf_est.at[iindex, 'ALS'] = None
   

outcome:

In [19]:
gdf_est.head()

,lat,lon,geometry,data,night,solar_el,polygon,ALS
0,6.608361e+06,551219.590202,POINT (551219.590 6608361.451),17.497229,1,-21.437059,NaN,None
1,6.608261e+06,551210.711176,POINT (551210.711 6608261.445),17.498577,1,-21.437450,"POLYGON ((551209.1918591695 6608312.177747834,...",None
2,6.608162e+06,551201.718180,POINT (551201.718 6608161.863),17.499891,1,-21.437843,"POLYGON ((551200.2384415013 6608212.187494098,...",None
3,6.608062e+06,551192.945760,POINT (551192.946 6608061.858),17.501179,1,-21.438231,"POLYGON ((551191.3718946896 6608112.578340041,...",None
4,6.607962e+06,551184.167062,POINT (551184.167 6607962.279),17.502453,1,-21.438620,"POLYGON ((551182.5780811904 6608012.578400162,...",None


Show only the ones that did have intersection: 

In [22]:
rslt_df = gdf_est[gdf_est['ALS'].notnull()] 
rslt_df

,lat,lon,geometry,data,night,solar_el,polygon,ALS
1354,6.589150e+06,529085.101864,POINT (529085.102 6589150.301),54.765408,0,10.044330,"POLYGON ((529237.7392106915 6588311.000257394,...",[2]
1355,6.589249e+06,529067.570673,POINT (529067.571 6589249.190),54.920410,0,10.043892,"POLYGON ((529082.2817386887 6589200.580703241,...",[2]
1356,6.589328e+06,529053.222983,POINT (529053.223 6589328.128),55.362183,0,10.043544,"POLYGON ((529064.5325691722 6589299.456737614,...",[2]
1357,6.589447e+06,529032.514147,POINT (529032.514 6589446.543),53.967079,0,10.043012,"POLYGON ((529050.5197762146 6589378.414465231,...",[2]
1358,6.589545e+06,529015.526640,POINT (529015.527 6589545.436),50.626347,0,10.042563,"POLYGON ((529029.9627021243 6589496.836636609,...",[2]
1359,6.589842e+06,528963.595332,POINT (528963.595 6589842.109),31.672342,0,10.041228,"POLYGON ((529012.815577011 6589595.721489933, ...","[2, 3]"
1360,6.589941e+06,528946.178415,POINT (528946.178 6589940.999),18.128252,0,10.040792,"POLYGON ((528960.8316923111 6589892.391491281,...",[3]
1361,6.590039e+06,528928.657394,POINT (528928.657 6590039.464),18.127047,0,10.040353,"POLYGON ((528943.3261102508 6589991.276897434,...",[3]


## Clipping
Ideally, if I know which ALS files an ICESat point intersects with, then I would do a sort of loop:
1. iterate over all icesat points
2. for each als file that it intersects with, clip it with the polygon which is the icesat points bbox - that gives the ALS transect
3. if there is an intersection with more than one als file, merge (?) them
4. using the clipped als point cloud, calculate the values needed (eg average height, canopy gap probability, etc) and save the values with the icesat point.

However I don't have that yet - here I just took one of the icesat bbox and tried clipping the ALS file.

In [23]:
# example polygon that I know intersects
pol = rslt_df.iloc[0]['polygon']
print(pol.wkt)

POLYGON ((529237.7392106915 6588311.000257394, 529225.9180542016 6588308.936217849, 529082.2817386887 6589200.580703241, 529070.4659804297 6589198.485982074, 529237.7392106915 6588311.000257394))


Saw an example of clipping here 
https://notebooks.githubusercontent.com/view/ipynb?browser=unknown_browser&color_mode=auto&commit=3a2bea6caaba3a2e8b79c50b355de53414233ab6&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f726f636b6573746174652f706f696e742d636c6f75642d70726f63657373696e672f336132626561366361616261336132653862373963353062333535646535333431343233336162362f6e6f7465626f6f6b732f706f696e742d636c6f75642d70726f63657373696e672e6970796e62&logged_in=false&nwo=rockestate%2Fpoint-cloud-processing&path=notebooks%2Fpoint-cloud-processing.ipynb&platform=unknown_platform&repository_id=107790126&repository_type=Repository&version=0

In [24]:
b = pol.bounds
lidar_filename = 'sambliku_1.las'
cropper = {
    "pipeline": [ '../Data/ALS/'+ lidar_filename,
        {   "type":"filters.crop",
            'bounds':str(([b[0], b[2]],[b[1], b[3]]))},
        {   "type":"filters.crop",
            'polygon':pol.wkt},
        #{   "type":"filters.hag"},
        {   "type":"filters.eigenvalues",
            "knn":16},
        {   "type":"filters.normal",
            "knn":16},
        {
            "type":"writers.las",
            "filename":"sambliku_cropped.las"
    }
    ]}
pipeline = pdal.Pipeline(json.dumps(cropper))
#pipeline.validate()
%time n_points = pipeline.execute()

CPU times: total: 4.05 s
Wall time: 4.09 s


And this gave me a clipped ALS point cloud. I don't think I can show it here, so you need to take my word for it.

## Conclusion

* I still have a lot of work to do with filtering the ICESat-2 data
* I am mostly concerned how to scale things. First, how to find which ALS files the ICESat points intersect with. I saw a discussion on spatial partitioning of the geodataframe - perhaps something along that line. Essentially I don't need to do intersection test between every ICESat pont and every ALS file, only the ones that have coordinates close to each other.
* I am relying on dataframes a lot, which is not something we used too much in our courses. I wonder if I am relying on external libraries too much.